# Optimization Data Analysis

## 0: Setup

### 0.1: Importing Packages and Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

In [ ]:
option_prices = pd.read_csv("option_pricing.csv")

### 0.2: Function Definitions

#### 0.2.1: Empirical Option Price

According to the Black-Scholes differential equation:

$$\frac{\partial V}{\partial t} + \frac{\sigma ^ 2 S ^ 2}{2} S^2 \frac{\partial^2 V}{\partial S^2} + r S \frac{\partial V}{\partial S} - r V = 0$$

which is solved/transformed into the following (for European call and put prices, respectively):

$$C(S, t) = S\Phi(d_1) - Ke^{-rt}\Phi(d_2)$$
$$P(S, t) = Ke^{-rt}\Phi(-d_2) - S\Phi(-d_1)$$

where $\Phi$ is the standard normal cdf and $d_1 = \frac{\ln\left(\frac{S}{K}\right) + (r + \frac{\sigma ^ 2}{2})T}{\sigma\sqrt{T}}$, $d_2 = d_1 - \sigma\sqrt{T}$.

In [ ]:
def empirical_option(row) -> float:
    # Setting variables based on the row input.
    # This would have been easier with just putting the functions in multiple parameter inputs, but this is non-trivial in Python
    S_0 = 100
    K = row["Strike Price"]
    r = row["Risk-Free Rate"]
    sigma = row["Volatility"]
    T = row["Expiry Time"]
    call_flag = row["Call Flag"]
    
    d_1 = (np.log(S_0 / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d_2 = d_1 - sigma * np.sqrt(T)
    
    # call price
    if call_flag == 1:
        return S_0 * norm.cdf(d_1) - K * np.exp(-r * T) * norm.cdf(d_2)
    # put price
    return K * np.exp(-r * T) * norm.cdf(-d_2) - S_0 * norm.cdf(-d_1)

### 0.2.2: Normalized $L^2$ Error

The pricing errors were calculated using the normalized $L^2$ error, which is defined by the following equation:

$$\frac{\left\lVert y - \hat{y}\right\rVert_2}{\lVert y\rVert_2}$$

where $\hat{y}$ is the predicted value by either the original or optimized pricing method and $y$ is the empirical price.

In [ ]:
# Implementation is ugly/elss generalizable with two function but is much easier in practice
def normalized_l2_error_original(row) -> float:
    return np.linalg.norm(row["Original Method Price"] - row["Empirical Price"]) / np.linalg.norm(row["Empirical Price"])
def normalized_l2_error_optimized(row) -> float:
    return np.linalg.norm(row["Optimized Method Price"] - row["Empirical Price"]) / np.linalg.norm(row["Empirical Price"])

## 1: Data Manipulation (adding and modifying columns)

### 1.1: Adding Empirical Price

In [ ]:
option_prices["Empirical Price"] = option_prices.apply(empirical_option, axis = 1)

### 1.2: Adding Original and Optimized Method Price Errors

In [ ]:
option_prices["Original Error"], option_prices["Optimized Error"] = option_prices.apply(normalized_l2_error_original, axis = 1), option_prices.apply(normalized_l2_error_optimized, axis = 1)

In [ ]:
option_prices.head()

## 2: Plots

I'm aware that I should have made a plotting function; it would have made this less of an eyesore. There are enough inputs to where it is simpler and faster to copy & paste the previous code and change one parameter than write a function for it. This is not generalizable, but it does not have to be.

I will omit analysis below each plot and summarize it in the final section for brevity purposes.

### 2.1: Data Organization

This could be more space/memory efficient; too many copies made for the sake of being explicit.

In [ ]:
power_calls, power_puts = option_prices.loc[(option_prices["Basis"] == 0) & (option_prices["Call Flag"] == 1)], option_prices.loc[(option_prices["Basis"] == 0) & (option_prices["Call Flag"] == 0)]
hermitian_calls, hermitian_puts = option_prices.loc[(option_prices["Basis"] == 1) & (option_prices["Call Flag"] == 1)], option_prices.loc[(option_prices["Basis"] == 1) & (option_prices["Call Flag"] == 0)]
laguerre_calls, laguerre_puts = option_prices.loc[(option_prices["Basis"] == 2) & (option_prices["Call Flag"] == 1)], option_prices.loc[(option_prices["Basis"] == 2) & (option_prices["Call Flag"] == 0)]

#### 2.2: Error vs Risk-Free Rates

Setting $\sigma = 0.3$ and $T = 1$. Feel free to change these values to draw your own conclusions.

##### 2.2.1: Calls

Setting $K = 110$.

In [ ]:
vol_temp = 0.3
expiry_temp = 1
strike_temp = 110

filtered_power_calls = power_calls.loc[(power_calls["Volatility"] == vol_temp) & (power_calls["Expiry Time"] == expiry_temp) & (power_calls["Strike Price"] == strike_temp)]
filtered_hermitian_calls = hermitian_calls.loc[(hermitian_calls["Volatility"] == vol_temp) & (hermitian_calls["Expiry Time"] == expiry_temp) & (hermitian_calls["Strike Price"] == strike_temp)]
filtered_laguerre_calls = laguerre_calls.loc[(laguerre_calls["Volatility"] == vol_temp) & (laguerre_calls["Expiry Time"] == expiry_temp) & (laguerre_calls["Strike Price"] == strike_temp)]

plt.plot(filtered_power_calls["Risk-Free Rate"], filtered_power_calls["Original Error"], color = 'blue', label = 'Original')
plt.plot(filtered_power_calls["Risk-Free Rate"], filtered_power_calls["Optimized Error"], color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_calls["Risk-Free Rate"], filtered_hermitian_calls["Optimized Error"], color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_calls["Risk-Free Rate"], filtered_laguerre_calls["Optimized Error"], color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Risk-Free Rate vs L^2 Error for Calls')
plt.xlabel("Risk-Free Rate")
plt.ylabel("Normalized L^2 Error")
plt.legend()
plt.grid(True)
plt.show()

##### 2.2.2: Puts

Setting $K = 90$.

In [ ]:
strike_temp = 90

filtered_power_puts = power_puts.loc[(power_puts["Volatility"] == vol_temp) & (power_puts["Expiry Time"] == expiry_temp) & (power_puts["Strike Price"] == strike_temp)]
filtered_hermitian_puts = hermitian_puts.loc[(hermitian_puts["Volatility"] == vol_temp) & (hermitian_puts["Expiry Time"] == expiry_temp) & (hermitian_puts["Strike Price"] == strike_temp)]
filtered_laguerre_puts = laguerre_puts.loc[(laguerre_puts["Volatility"] == vol_temp) & (laguerre_puts["Expiry Time"] == expiry_temp) & (laguerre_puts["Strike Price"] == strike_temp)]

plt.plot(filtered_power_puts["Risk-Free Rate"], filtered_power_puts["Original Error"], color = 'blue', label = 'Original')
plt.plot(filtered_power_puts["Risk-Free Rate"], filtered_power_puts["Optimized Error"], color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_puts["Risk-Free Rate"], filtered_hermitian_puts["Optimized Error"], color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_puts["Risk-Free Rate"], filtered_laguerre_puts["Optimized Error"], color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Risk-Free Rate vs L^2 Error for Puts')
plt.xlabel("Risk-Free Rate")
plt.ylabel("Normalized L^2 Error")
plt.legend()
plt.grid(True)
plt.show()

#### 2.3: Error vs Volatility

Setting $r = 0.05$ and $T = 1$.

##### 2.3.1: Calls

Setting $K = 110$.

In [ ]:
risk_temp = 0.05
strike_temp = 110
expiry_temp = 1

filtered_power_calls = power_calls.loc[(power_calls["Risk-Free Rate"] == risk_temp) & (power_calls["Expiry Time"] == expiry_temp) & (power_calls["Strike Price"] == strike_temp)]
filtered_hermitian_calls = hermitian_calls.loc[(hermitian_calls["Risk-Free Rate"] == risk_temp) & (hermitian_calls["Expiry Time"] == expiry_temp) & (hermitian_calls["Strike Price"] == strike_temp)]
filtered_laguerre_calls = laguerre_calls.loc[(laguerre_calls["Risk-Free Rate"] == risk_temp) & (laguerre_calls["Expiry Time"] == expiry_temp) & (laguerre_calls["Strike Price"] == strike_temp)]

plt.plot(filtered_power_calls["Volatility"], filtered_power_calls["Original Error"], color = 'blue', label = 'Original')
plt.plot(filtered_power_calls["Volatility"], filtered_power_calls["Optimized Error"], color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_calls["Volatility"], filtered_hermitian_calls["Optimized Error"], color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_calls["Volatility"], filtered_laguerre_calls["Optimized Error"], color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Volatility vs L^2 Error for Calls')
plt.xlabel("Volatility")
plt.ylabel("Normalized L^2 Error")
plt.legend()
plt.grid(True)
plt.show()

##### 2.3.2: Puts

Setting $K = 90$.

In [ ]:
strike_temp = 90

filtered_power_puts = power_puts.loc[(power_puts["Risk-Free Rate"] == risk_temp) & (power_puts["Expiry Time"] == expiry_temp) & (power_puts["Strike Price"] == strike_temp)]
filtered_hermitian_puts = hermitian_puts.loc[(hermitian_puts["Risk-Free Rate"] == risk_temp) & (hermitian_puts["Expiry Time"] == expiry_temp) & (hermitian_puts["Strike Price"] == strike_temp)]
filtered_laguerre_puts = laguerre_puts.loc[(laguerre_puts["Risk-Free Rate"] == risk_temp) & (laguerre_puts["Expiry Time"] == expiry_temp) & (laguerre_puts["Strike Price"] == strike_temp)]

plt.plot(filtered_power_puts["Volatility"], filtered_power_puts["Original Error"], color = 'blue', label = 'Original')
plt.plot(filtered_power_puts["Volatility"], filtered_power_puts["Optimized Error"], color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_puts["Volatility"], filtered_hermitian_puts["Optimized Error"], color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_puts["Volatility"], filtered_laguerre_puts["Optimized Error"], color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Volatility vs L^2 Error for Puts')
plt.xlabel("Volatility")
plt.ylabel("Normalized L^2 Error")
plt.legend()
plt.grid(True)
plt.show()

#### 2.4: Error vs Strike Price

Setting $r = 0.05$, $T = 1$ and $\sigma = 0.3$.

##### 2.4.1: Calls

In [ ]:
risk_temp = 0.05
expiry_temp = 1
vol_temp = 0.3

filtered_power_calls = power_calls.loc[(power_calls["Risk-Free Rate"] == risk_temp) & (power_calls["Expiry Time"] == expiry_temp) & (power_calls["Volatility"] == vol_temp)]
filtered_hermitian_calls = hermitian_calls.loc[(hermitian_calls["Risk-Free Rate"] == risk_temp) & (hermitian_calls["Expiry Time"] == expiry_temp) & (hermitian_calls["Volatility"] == vol_temp)]
filtered_laguerre_calls = laguerre_calls.loc[(laguerre_calls["Risk-Free Rate"] == risk_temp) & (laguerre_calls["Expiry Time"] == expiry_temp) & (laguerre_calls["Volatility"] == vol_temp)]

plt.plot(filtered_power_calls["Strike Price"], filtered_power_calls["Original Error"], color = 'blue', label = 'Original')
plt.plot(filtered_power_calls["Strike Price"], filtered_power_calls["Optimized Error"], color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_calls["Strike Price"], filtered_hermitian_calls["Optimized Error"], color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_calls["Strike Price"], filtered_laguerre_calls["Optimized Error"], color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Strike Price vs L^2 Error for Calls')
plt.xlabel("Strike Price")
plt.ylabel("Normalized L^2 Error")
plt.legend()
plt.grid(True)
plt.show()

##### 2.4.2: Puts

This must be a systematic issue since normalized error is very high for both methds/all bases.

In [ ]:
filtered_power_puts = power_puts.loc[(power_puts["Risk-Free Rate"] == risk_temp) & (power_puts["Expiry Time"] == expiry_temp) & (power_puts["Volatility"] == vol_temp)]
filtered_hermitian_puts = hermitian_puts.loc[(hermitian_puts["Risk-Free Rate"] == risk_temp) & (hermitian_puts["Expiry Time"] == expiry_temp) & (hermitian_puts["Volatility"] == vol_temp)]
filtered_laguerre_puts = laguerre_puts.loc[(laguerre_puts["Risk-Free Rate"] == risk_temp) & (laguerre_puts["Expiry Time"] == expiry_temp) & (laguerre_puts["Volatility"] == vol_temp)]

plt.plot(filtered_power_puts["Strike Price"], filtered_power_puts["Original Error"], color = 'blue', label = 'Original')
plt.plot(filtered_power_puts["Strike Price"], filtered_power_puts["Optimized Error"], color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_puts["Strike Price"], filtered_hermitian_puts["Optimized Error"], color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_puts["Strike Price"], filtered_laguerre_puts["Optimized Error"], color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Strike Price vs L^2 Error for Puts')
plt.xlabel("Strike Price")
plt.ylabel("Normalized L^2 Error")
plt.legend()
plt.grid(True)
plt.show()

#### 2.5: Error vs Expiry Time

Setting $r = 0.05$ and $\sigma = 0.3$.

##### 2.5.1: Calls

$K = 110$.

In [ ]:
risk_temp = 0.05
strike_temp = 110
vol_temp = 0.3

filtered_power_calls = power_calls.loc[(power_calls["Risk-Free Rate"] == risk_temp) & (power_calls["Strike Price"] == strike_temp) & (power_calls["Volatility"] == vol_temp)]
filtered_hermitian_calls = hermitian_calls.loc[(hermitian_calls["Risk-Free Rate"] == risk_temp) & (hermitian_calls["Strike Price"] == strike_temp) & (hermitian_calls["Volatility"] == vol_temp)]
filtered_laguerre_calls = laguerre_calls.loc[(laguerre_calls["Risk-Free Rate"] == risk_temp) & (laguerre_calls["Strike Price"] == strike_temp) & (laguerre_calls["Volatility"] == vol_temp)]

plt.plot(filtered_power_calls["Expiry Time"], filtered_power_calls["Original Error"], color = 'blue', label = 'Original')
plt.plot(filtered_power_calls["Expiry Time"], filtered_power_calls["Optimized Error"], color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_calls["Expiry Time"], filtered_hermitian_calls["Optimized Error"], color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_calls["Expiry Time"], filtered_laguerre_calls["Optimized Error"], color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Expiry Time vs L^2 Error for Calls')
plt.xlabel("Expiry Time")
plt.ylabel("Normalized L^2 Error")
plt.legend()
plt.grid(True)
plt.show()

##### 2.5.2: Puts

$K = 90$.

In [ ]:
strike_temp = 90

filtered_power_puts = power_puts.loc[(power_puts["Risk-Free Rate"] == risk_temp) & (power_puts["Strike Price"] == strike_temp) & (power_puts["Volatility"] == vol_temp)]
filtered_hermitian_puts = hermitian_puts.loc[(hermitian_puts["Risk-Free Rate"] == risk_temp) & (hermitian_puts["Strike Price"] == strike_temp) & (hermitian_puts["Volatility"] == vol_temp)]
filtered_laguerre_puts = laguerre_puts.loc[(laguerre_puts["Risk-Free Rate"] == risk_temp) & (laguerre_puts["Strike Price"] == strike_temp) & (laguerre_puts["Volatility"] == vol_temp)]

plt.plot(filtered_power_puts["Expiry Time"], filtered_power_puts["Original Error"], color = 'blue', label = 'Original')
plt.plot(filtered_power_puts["Expiry Time"], filtered_power_puts["Optimized Error"], color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_puts["Expiry Time"], filtered_hermitian_puts["Optimized Error"], color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_puts["Expiry Time"], filtered_laguerre_puts["Optimized Error"], color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Expiry Time vs L^2 Error for Puts')
plt.xlabel("Expiry Time")
plt.ylabel("Normalized L^2 Error")
plt.legend()
plt.grid(True)
plt.show()

### 2.6: Computation Time vs Risk-Free Rate

Setting $\sigma = 0.3$ and $T = 1$.

#### 2.6.1: Calls

$K = 110$.

In [ ]:
strike_temp = 110
vol_temp = 0.3
expiry_temp = 1

filtered_power_calls = power_calls.loc[(power_calls["Strike Price"] == strike_temp) & (power_calls["Volatility"] == vol_temp) & (power_calls["Expiry Time"] == expiry_temp)]
filtered_hermitian_calls = hermitian_calls.loc[(hermitian_calls["Strike Price"] == strike_temp) & (hermitian_calls["Volatility"] == vol_temp) & (hermitian_calls["Expiry Time"] == expiry_temp)]
filtered_laguerre_calls = laguerre_calls.loc[(laguerre_calls["Strike Price"] == strike_temp) & (laguerre_calls["Volatility"] == vol_temp) & (laguerre_calls["Expiry Time"] == expiry_temp)]

# Divide optimized computation time by 3 since parallelization
plt.plot(filtered_power_calls["Risk-Free Rate"], filtered_power_calls["Original Computation Time"], color = 'blue', label = 'Original')
plt.plot(filtered_power_calls["Risk-Free Rate"], filtered_power_calls["Optimized Computation Time"] / 3, color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_calls["Risk-Free Rate"], filtered_hermitian_calls["Optimized Computation Time"] / 3, color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_calls["Risk-Free Rate"], filtered_laguerre_calls["Optimized Computation Time"] / 3, color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Risk-Free Rate vs Computation Time for Calls')
plt.xlabel("Risk-Free Rate")
plt.ylabel("Computation Time (seconds)")
plt.legend()
plt.grid(True)
plt.show()

#### 2.6.2: Puts

$K = 90$.

In [ ]:
strike_temp = 90

filtered_power_puts = power_puts.loc[(power_puts["Strike Price"] == strike_temp) & (power_puts["Volatility"] == vol_temp) & (power_puts["Expiry Time"] == expiry_temp)]
filtered_hermitian_puts = hermitian_puts.loc[(hermitian_puts["Strike Price"] == strike_temp) & (hermitian_puts["Volatility"] == vol_temp) & (hermitian_puts["Expiry Time"] == expiry_temp)]
filtered_laguerre_puts = laguerre_puts.loc[(laguerre_puts["Strike Price"] == strike_temp) & (laguerre_puts["Volatility"] == vol_temp) & (laguerre_puts["Expiry Time"] == expiry_temp)]

# Divide optimized computation time by 3 since parallelization
plt.plot(filtered_power_puts["Risk-Free Rate"], filtered_power_puts["Original Computation Time"], color = 'blue', label = 'Original')
plt.plot(filtered_power_puts["Risk-Free Rate"], filtered_power_puts["Optimized Computation Time"] / 3, color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_puts["Risk-Free Rate"], filtered_hermitian_puts["Optimized Computation Time"] / 3, color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_puts["Risk-Free Rate"], filtered_laguerre_puts["Optimized Computation Time"] / 3, color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Risk-Free Rate vs Computation Time for Puts')
plt.xlabel("Risk-Free Rate")
plt.ylabel("Computation Time (seconds)")
plt.legend()
plt.grid(True)
plt.show()

### 2.7: Computation Time vs Volatility

Setting $r = 0.05$ and $T = 1$.

#### 2.7.1: Calls

$K = 110$.

In [ ]:
strike_temp = 110

filtered_power_calls = power_calls.loc[(power_calls["Strike Price"] == strike_temp) & (power_calls["Risk-Free Rate"] == risk_temp) & (power_calls["Expiry Time"] == expiry_temp)]
filtered_hermitian_calls = hermitian_calls.loc[(hermitian_calls["Strike Price"] == strike_temp) & (hermitian_calls["Risk-Free Rate"] == risk_temp) & (hermitian_calls["Expiry Time"] == expiry_temp)]
filtered_laguerre_calls = laguerre_calls.loc[(laguerre_calls["Strike Price"] == strike_temp) & (laguerre_calls["Risk-Free Rate"] == risk_temp) & (laguerre_calls["Expiry Time"] == expiry_temp)]

# Divide optimized computation time by 3 since parallelization
plt.plot(filtered_power_calls["Volatility"], filtered_power_calls["Original Computation Time"], color = 'blue', label = 'Original')
plt.plot(filtered_power_calls["Volatility"], filtered_power_calls["Optimized Computation Time"] / 3, color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_calls["Volatility"], filtered_hermitian_calls["Optimized Computation Time"] / 3, color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_calls["Volatility"], filtered_laguerre_calls["Optimized Computation Time"] / 3, color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Volatility vs Computation Time for Calls')
plt.xlabel("Volatility")
plt.ylabel("Computation Time (seconds)")
plt.legend()
plt.grid(True)
plt.show()

#### 2.7.2: Puts

$K = 90$.

In [ ]:
strike_temp = 90

filtered_power_puts = power_puts.loc[(power_puts["Strike Price"] == strike_temp) & (power_puts["Risk-Free Rate"] == risk_temp) & (power_puts["Expiry Time"] == expiry_temp)]
filtered_hermitian_puts = hermitian_puts.loc[(hermitian_puts["Strike Price"] == strike_temp) & (hermitian_puts["Risk-Free Rate"] == risk_temp) & (hermitian_puts["Expiry Time"] == expiry_temp)]
filtered_laguerre_puts = laguerre_puts.loc[(laguerre_puts["Strike Price"] == strike_temp) & (laguerre_puts["Risk-Free Rate"] == risk_temp) & (laguerre_puts["Expiry Time"] == expiry_temp)]

# Divide optimized computation time by 3 since parallelization
plt.plot(filtered_power_puts["Volatility"], filtered_power_puts["Original Computation Time"], color = 'blue', label = 'Original')
plt.plot(filtered_power_puts["Volatility"], filtered_power_puts["Optimized Computation Time"] / 3, color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_puts["Volatility"], filtered_hermitian_puts["Optimized Computation Time"] / 3, color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_puts["Volatility"], filtered_laguerre_puts["Optimized Computation Time"] / 3, color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Volatility vs Computation Time for Calls')
plt.xlabel("Volatility")
plt.ylabel("Computation Time (seconds)")
plt.legend()
plt.grid(True)
plt.show()

### 2.8: Computation Time vs Strike Price

Setting $r = 0.05, \sigma = 0.3,$ and $T = 1$.

#### 2.8.1: Calls

In [ ]:
filtered_power_calls = power_calls.loc[(power_calls["Volatility"] == vol_temp) & (power_calls["Risk-Free Rate"] == risk_temp) & (power_calls["Expiry Time"] == expiry_temp)]
filtered_hermitian_calls = hermitian_calls.loc[(hermitian_calls["Volatility"] == vol_temp) & (hermitian_calls["Risk-Free Rate"] == risk_temp) & (hermitian_calls["Expiry Time"] == expiry_temp)]
filtered_laguerre_calls = laguerre_calls.loc[(laguerre_calls["Volatility"] == vol_temp) & (laguerre_calls["Risk-Free Rate"] == risk_temp) & (laguerre_calls["Expiry Time"] == expiry_temp)]

# Divide optimized computation time by 3 since parallelization
plt.plot(filtered_power_calls["Strike Price"], filtered_power_calls["Original Computation Time"], color = 'blue', label = 'Original')
plt.plot(filtered_power_calls["Strike Price"], filtered_power_calls["Optimized Computation Time"] / 3, color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_calls["Strike Price"], filtered_hermitian_calls["Optimized Computation Time"] / 3, color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_calls["Strike Price"], filtered_laguerre_calls["Optimized Computation Time"] / 3, color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Strike Price vs Computation Time for Calls')
plt.xlabel("Strike Price")
plt.ylabel("Computation Time (seconds)")
plt.legend()
plt.grid(True)
plt.show()

#### 2.8.2: Puts

In [ ]:
filtered_power_puts = power_puts.loc[(power_puts["Volatility"] == vol_temp) & (power_puts["Risk-Free Rate"] == risk_temp) & (power_puts["Expiry Time"] == expiry_temp)]
filtered_hermitian_puts = hermitian_puts.loc[(hermitian_puts["Volatility"] == vol_temp) & (hermitian_puts["Risk-Free Rate"] == risk_temp) & (hermitian_puts["Expiry Time"] == expiry_temp)]
filtered_laguerre_puts = laguerre_puts.loc[(laguerre_puts["Volatility"] == vol_temp) & (laguerre_puts["Risk-Free Rate"] == risk_temp) & (laguerre_puts["Expiry Time"] == expiry_temp)]

# Divide optimized computation time by 3 since parallelization
plt.plot(filtered_power_puts["Strike Price"], filtered_power_puts["Original Computation Time"], color = 'blue', label = 'Original')
plt.plot(filtered_power_puts["Strike Price"], filtered_power_puts["Optimized Computation Time"] / 3, color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_puts["Strike Price"], filtered_hermitian_puts["Optimized Computation Time"] / 3, color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_puts["Strike Price"], filtered_laguerre_puts["Optimized Computation Time"] / 3, color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Strike Price vs Computation Time for Puts')
plt.xlabel("Strike Price")
plt.ylabel("Computation Time (seconds)")
plt.legend()
plt.grid(True)
plt.show()

### 2.8: Computation Time vs Expiry Time   

Setting $r = 0.05$ and $\sigma = 0.3$.

#### 2.8.1: Calls

$K = 110$.

In [ ]:
strike_temp = 110

filtered_power_calls = power_calls.loc[(power_calls["Volatility"] == vol_temp) & (power_calls["Risk-Free Rate"] == risk_temp) & (power_calls["Strike Price"] == strike_temp)]
filtered_hermitian_calls = hermitian_calls.loc[(hermitian_calls["Volatility"] == vol_temp) & (hermitian_calls["Risk-Free Rate"] == risk_temp) & (hermitian_calls["Strike Price"] == strike_temp)]
filtered_laguerre_calls = laguerre_calls.loc[(laguerre_calls["Volatility"] == vol_temp) & (laguerre_calls["Risk-Free Rate"] == risk_temp) & (laguerre_calls["Strike Price"] == strike_temp)]

# Divide optimized computation time by 3 since parallelization
plt.plot(filtered_power_calls["Expiry Time"], filtered_power_calls["Original Computation Time"], color = 'blue', label = 'Original')
plt.plot(filtered_power_calls["Expiry Time"], filtered_power_calls["Optimized Computation Time"] / 3, color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_calls["Expiry Time"], filtered_hermitian_calls["Optimized Computation Time"] / 3, color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_calls["Expiry Time"], filtered_laguerre_calls["Optimized Computation Time"] / 3, color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Expiry Time vs Computation Time for Calls')
plt.xlabel("Expiry Time")
plt.ylabel("Computation Time (seconds)")
plt.legend()
plt.grid(True)
plt.show()

#### 2.8.2: Puts

$K = 90$.

In [ ]:
strike_temp = 90

filtered_power_puts = power_puts.loc[(power_puts["Volatility"] == vol_temp) & (power_puts["Risk-Free Rate"] == risk_temp) & (power_puts["Strike Price"] == strike_temp)]
filtered_hermitian_puts = hermitian_puts.loc[(hermitian_puts["Volatility"] == vol_temp) & (hermitian_puts["Risk-Free Rate"] == risk_temp) & (hermitian_puts["Strike Price"] == strike_temp)]
filtered_laguerre_puts = laguerre_puts.loc[(laguerre_puts["Volatility"] == vol_temp) & (laguerre_puts["Risk-Free Rate"] == risk_temp) & (laguerre_puts["Strike Price"] == strike_temp)]

# Divide optimized computation time by 3 since parallelization
plt.plot(filtered_power_puts["Expiry Time"], filtered_power_puts["Original Computation Time"], color = 'blue', label = 'Original')
plt.plot(filtered_power_puts["Expiry Time"], filtered_power_puts["Optimized Computation Time"] / 3, color = 'red', label = 'Power Optimized')
plt.plot(filtered_hermitian_puts["Expiry Time"], filtered_hermitian_puts["Optimized Computation Time"] / 3, color = 'orange', label = 'Hermitian Optimized')
plt.plot(filtered_laguerre_puts["Expiry Time"], filtered_laguerre_puts["Optimized Computation Time"] / 3, color = 'green', label = 'Laguerre Optimized')

plt.title('A Plot of Expiry Time vs Computation Time for Calls')
plt.xlabel("Expiry Time")
plt.ylabel("Computation Time (seconds)")
plt.legend()
plt.grid(True)
plt.show()